In [2]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

def scrape():
    url = 'https://www.example.com'
    response = requests.get(url)
    # print(response.content)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.select_one('p').text
    # pprint(soup)
    print(title)

In [3]:
import requests

url = "https://tf.tfrrs.org/lists/5352/2025_2026_NCAA_Division_I_Indoor_Qualifying_List?gender=m"

# 1. Define Headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",  # Tells the server you can accept JSON
    "Referer": "https://www.tfrrs.org/", # Tells the server you came from their main site
}

# 2. Pass the headers argument into the request
response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Success!")
    # data = response.json() # Since this is an API, use .json(), not BeautifulSoup
    # print(data)
else:
    print(f"Error: {response.status_code}")

Success!


In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")

# 1. Find the "Event Titles" first. 
# In your HTML, every event starts with a div class="custom-table-title"
event_headers = soup.find_all("div", class_="custom-table-title")

data = []

for header in event_headers:
    # 2. Extract the Event Name (e.g., "Mile") from the <h3> tag inside the header
    event_name = header.find("h3").get_text(strip=True)
    
    # 3. Find the specific list of results for THIS event
    # In the HTML, the results list is the immediate neighbor (sibling) of the title
    results_table = header.find_next_sibling("div", class_="performance-list")
    
    if results_table:
        # 4. Now find rows ONLY inside this specific event's table
        rows = results_table.find_all("div", class_="performance-list-row")
        
        for row in rows:
            athlete_div = row.find("div", attrs={"data-label": "Athlete"})
            time_div = row.find("div", attrs={"data-label": "Time"})
            
            athlete_name = athlete_div.get_text(strip=True) if athlete_div else "N/A"
            time_text = time_div.get_text(strip=True) if time_div else "N/A"
            
            # Print to verify
            print(f"Event: {event_name} | Athlete: {athlete_name} | Time: {time_text}")
            
            # Save properly
            data.append({
                "Event": event_name,
                "Athlete": athlete_name,
                "Time": time_text
            })



Event: 60 Meters | Athlete: Howell, Davonte | Time: 6.49
Event: 60 Meters | Athlete: Folson, Traunard | Time: 6.49
Event: 60 Meters | Athlete: McCallum, T'Mars | Time: 6.59
Event: 60 Meters | Athlete: Fahmi, Azeem | Time: 6.60
Event: 60 Meters | Athlete: Brown, Kyler | Time: 6.60
Event: 60 Meters | Athlete: Clunis, Stephen | Time: 6.61
Event: 60 Meters | Athlete: Lawerence, Darien | Time: 6.62
Event: 60 Meters | Athlete: John, Nazzio | Time: 6.62
Event: 60 Meters | Athlete: Odamtten, Medwin | Time: 6.63
Event: 60 Meters | Athlete: Hood, Jr, Casey | Time: 6.63
Event: 60 Meters | Athlete: Finley, Jayden | Time: 6.64
Event: 60 Meters | Athlete: Simusialela, Denzel | Time: 6.64
Event: 60 Meters | Athlete: Ancrum, Nigel | Time: 6.64
Event: 60 Meters | Athlete: Rosario, Miguel | Time: 6.64@
Event: 60 Meters | Athlete: Jones, Miles | Time: 6.65
Event: 60 Meters | Athlete: Sylvester, Jared | Time: 6.65
Event: 60 Meters | Athlete: Lewis, Omari | Time: 6.66
Event: 60 Meters | Athlete: Rogers, Ja

In [7]:
import pandas as pd
df = pd.DataFrame(data)
display(df)

,Event,Athlete,Time
0,60 Meters,"Howell, Davonte",6.49
1,60 Meters,"Folson, Traunard",6.49
2,60 Meters,"McCallum, T'Mars",6.59
3,60 Meters,"Fahmi, Azeem",6.60
4,60 Meters,"Brown, Kyler",6.60
...,...,...,...
842,Heptathlon,"Clark, Andrew",N/A
843,Heptathlon,"Thai, Gabriel",N/A
844,Heptathlon,"O'Regan, Samuel",N/A
845,Heptathlon,"Crowley, Ian",N/A


In [ ]:
# Now you can group by the 'Event' column normally
df_merged = df.groupby('Event')

type(df_merged)


pandas.core.groupby.generic.DataFrameGroupBy

In [ ]:
# display(df.loc["60 Meters",:])    

for event in df_merged:
    print("i")


# display(df_merged)

# with pd.option_context('display.max_rows', 1000):
#     display(df)




i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i


In [65]:
# Unpack the tuple immediately for clarity
for event_name, event_df in df_merged:
    print(f"--- {event_name} ---")
    
    # We use .iloc to look at positions 0 to 16
    # We slice [:16] so it doesn't crash if an event has fewer than 16 people
    top_16 = event_df.iloc[:16]
    
    # Iterate through these top rows
    # enumerate helps us get the rank number (i) starting at 1
    for i, (index, row) in enumerate(top_16.iterrows(), start=1):
        
        # Access columns by their actual names
        athlete_name = row['Athlete']  
        time = row['Time']
        
        print(f"{i}. {athlete_name} - {time}")
        

--- 200 Meters ---
1. Dossman, Ian - 20.64@
2. Johnson, Jalen - 20.76
3. Collins, Kevin - 20.85
4. Rosario, Miguel - 20.88@
5. Clarke, Jackson - 20.97
6. Oyetunji, Ayomide - 20.98
7. Benjamin, Nakhi - 21.05
8. Simusialela, Denzel - 21.06
9. Schlotterbeck, Cole - 21.08
10. Johnson, Ian - 21.10
11. Harper, Dominic - 21.12
12. Jordan, Aljernun (AJ) - 21.12
13. Amadi, Zion - 21.13
14. Pitts Jr., Sedgwrick - 21.14
15. Lawerence, Darien - 21.21
16. Gant, Joey - 21.21
--- 3000 Meters ---
1. Sahlman, Colin - 7:36.71
2. Couttie, George - 7:36.74
3. Birnbaum, Simeon - 7:39.65
4. Er Raouy, Taha - 7:40.56
5. Cook, Elliott - 7:41.57
6. Hamlin, Jesse - 7:43.06
7. Reina, Reuben - 7:43.16
8. Serem, Dominic - 7:45.61
9. Balazs, Benjamin - 7:47.65
10. Jenkins, Noah - 7:49.58#@
11. Kipyego, Devan - 7:51.58
12. Khchina, Jaouad - 7:51.76
13. Westphal, Thomas - 7:52.18
14. Givens, Isaiah - 7:53.07
15. Clayton, Jason - 7:54.17
16. Kiplagat, Sila - 7:54.37#@
--- 4 x 400 Relay ---
1. N/A - 3:04.39
2. N/A - 3:0